In [1]:
%matplotlib widget

In [2]:
from functools import partial

In [3]:
from libschrodinger import *

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
from pathlib import Path

In [7]:
from typing import Dict

In [8]:
caseName : str = "test_option_6"

In [9]:
math = np

In [10]:
spatialStep : float = .01
length : float = 4
temporalStep : float = (spatialStep ** 2) / 4
courantNumber : float = 1.0
pointCount : int = int(length / spatialStep)
potentialHeight : float = 1000
preRegionLength = .3
preRegionScalar = 2
packetStandardDeviation : float = preRegionLength / preRegionScalar
packetStartX : float = 1 - ((preRegionScalar / 2) * packetStandardDeviation)
packetStartY : float = .5
packetWaveNumber : float = 13 * np.pi
barrierCount : int = 3
postRegionLength = .3
intraRegionLength : float = 1 - (postRegionLength + preRegionLength)
barrierWidth : float = intraRegionLength / barrierCount
constantPotentialLengthRatios : List[float] = [postRegionLength] \
        + ([barrierWidth] * barrierCount) \
        + [preRegionLength]
print("Point Count: ", pointCount)
print("Post Region Length: ", postRegionLength)
print("Intra Region Length: ", intraRegionLength)
print("Pre Region Length: ", preRegionLength)
print("Barrier Length: ", barrierWidth)
print("Total Length: ", intraRegionLength + postRegionLength + preRegionLength)
print("Total Length Ratios: ", np.sum(constantPotentialLengthRatios))
frameCount : int = 60

Point Count:  400
Post Region Length:  0.3
Intra Region Length:  0.4
Pre Region Length:  0.3
Barrier Length:  0.13333333333333333
Total Length:  1.0
Total Length Ratios:  1.0


In [11]:
wavePacketFunction = lambda position : makeWavePacket(
            position, 
            packetStartX * length, 
            packetStartY * length, 
            spatialStep, 
            packetStandardDeviation, 
            packetWaveNumber, 
            math
        )

In [12]:
potentials : List[List[float]] = [
        [0, 0, 1, 1, 1], 
    ]

In [13]:
spatialStep ** 2

0.0001

In [14]:
profiles : List[SimulationProfile] = constantSimulationProfiles(
        wavePacketFunction, 
        spatialStep, 
        temporalStep, 
        length, 
        constantPotentialLengthRatios, 
        potentials, 
        potentialHeight, 
        pointCount, 
        True, 
        edgeBound = True, 
        logFunction = performenceLog, 
        gpuAccelerated = True, 
        defaultMatrixSolveMethod = lambda profile, operator, independantTerms, previous \
                : solveMatrixApproximate(profile, operator, independantTerms, previous, 1e-5)
    )

In [15]:
constantPotentialLengthRatios

[0.3, 0.13333333333333333, 0.13333333333333333, 0.13333333333333333, 0.3]

In [16]:
allData, _, logs = recordConstantRegionSimulations(
        profiles, 
        frameCount, 
        caseName, 
        constantPotentialLengthRatios, 
        True, 
        constantRegionLabels = ["Post", "3rdStair", "2ndStair", "1stStair", "Pre"], 
        showBar = True, 
        showFPS = True, 
        showTotalTime = True
    )

[====================================================================================================]
[

TypeError: <lambda>() missing 1 required positional argument: 'previous'

In [ ]:
allData["packetStartX"] = packetStartX
allData["packetStartY"] = packetStartY
allData["initialWaveNumber"] = packetWaveNumber
allData["standardDeviation"] = packetStandardDeviation

In [ ]:
for key in allData: 
    allData[key] = asNumPyArray(allData[key])

In [ ]:
pd.DataFrame(allData).to_csv(str(Path.cwd() / caseName / (caseName + ".csv")))

In [ ]:
keys = list(logs[0].keys())
for key in keys: 
    print(len(logs[0][key]))
keys

In [ ]:
logKeyList = [
    'Starting "Step"',
    'Created knownStepMatrix',
    'Created unknownStepMatrix',
    'Started "Compute"',
    'Reshaped Wave Function',
    'Matrix Multiplication',
    'Solved For Independant Terms and Reshaped',
    'Appended Wave Function and Finished "Compute"',
    'Computed and Finished "Step"'
]

In [ ]:
copiedLogs = []
logIndex = 0
logPath = Path.cwd() / caseName / "logs"
logPath.mkdir(exist_ok = True)
for log in logs: 
    logCopy = {}
    for key in logKeyList: 
        logCopy[key] = log[key]
    copiedLogs.append(logCopy)
    pd.DataFrame(logCopy).to_csv(str(logPath / (caseName + "_log_" + str(logIndex) + ".csv")))
    logIndex += 1

In [ ]:
currentLog = copiedLogs[0]
plt.figure()
for stepLabel in logKeyList: 
    print(stepLabel)
    accelerated = np.array(currentLog[stepLabel])
    plt.plot(accelerated[1:] - accelerated[:-1], label=stepLabel)
plt.legend()

In [ ]:
currentLog = copiedLogs[0]
plt.figure()
plt.title("CPU (numpy) Delta Time Between Steps vs. Steps")
for stepLabel, nextLabel in  list(zip(logKeyList[:-1], logKeyList[1:])): 
    label = stepLabel + " -> " + nextLabel
    print(label)
    acceleratedCurrent = np.array(currentLog[stepLabel])
    acceleratedNext = np.array(currentLog[nextLabel])
    plt.plot(acceleratedNext - acceleratedCurrent, label=label)
plt.legend()